<a href="https://colab.research.google.com/github/SanjaghRecommand/Recommend/blob/Foad/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM
import pandas as pd
import tensorflow as tf
import torch
import pickle
from tabulate import tabulate


In [2]:
df = pd.read_csv ('/content/drive/MyDrive/Colab Notebooks/sanjaghshare/data/taskfullshare.csv')
df.drop(columns=['Unnamed: 0'], inplace=True)
# df2=df.sort_values('user').iloc[0:20000,]

In [3]:
mininit= df.Init.min()
df['time']=df.Init.apply(lambda x: round((x- mininit)/2000000000))
df.drop(columns='Init', inplace=True)
df= df[df['user'] >5 ]
df3=df


In [100]:
max_len= 10 # len(tf.keras.utils.pad_sequences(seqservice)[0])
item_vocab_size= 588

In [ ]:
seqservice , seqtime ,seqtarget  =[] ,[] ,[] 

for user in df.user.unique():
  user_data = df[df['user'] == user].sort_values('time')
  seqservice.append(user_data.service.values.tolist()[0:-1])
  seqtime.append(user_data.time.values.tolist()[0:-1])
  seqtarget.append(user_data.time.values.tolist()[1:])

  # print(user_data)
  # break
train_dict = dict(zip(['item_id', 'nb_days', 'target'],[tf.keras.utils.pad_sequences(seqservice, maxlen=max_len), tf.keras.utils.pad_sequences(seqtime, maxlen=max_len), tf.keras.utils.pad_sequences(seqtarget, maxlen=max_len)]))
# save dictionary to person_data.pkl file
with open('/content/drive/MyDrive/Colab Notebooks/sanjaghshare/data/train_dict10.pkl', 'wb') as fp:
    pickle.dump(train_dict, fp)
    print('dictionary saved successfully to file')

In [4]:
with open('/content/drive/MyDrive/Colab Notebooks/sanjaghshare/data/train_dict.pkl', 'rb') as fp:
    train_dict = pickle.load(fp)
    print(' dictionary loaded')


 dictionary loaded


In [5]:
#@title Create_train
def create_train_tfdata(train_feat_dict, train_target_tensor,    batch_size, buffer_size=None):
    """
    Create train tf dataset for model train input
    :param train_feat_dict: dict, containing the features tensors for train data
    :param train_target_tensor: np.array(), the training TARGET tensor
    :param batch_size: (int) size of the batch to work with
    :param buffer_size: (int) Optional. Default is None. Size of the buffer
    :return: (tuple) 1st element is the training dataset,
                     2nd is the number of steps per epoch (based on batch size)
    """
    if buffer_size is None:
        buffer_size = batch_size*50

    train_steps_per_epoch = len(train_target_tensor) // batch_size
    train_dataset = tf.data.Dataset.from_tensor_slices((train_feat_dict,train_target_tensor)).cache()
    train_dataset = train_dataset.shuffle(buffer_size).batch(batch_size)
    train_dataset = train_dataset.repeat().prefetch(tf.data.experimental.AUTOTUNE)
    
    return train_dataset, train_steps_per_epoch
  
train_feat_dict = {'item_id': train_dict['item_id'],'nb_days': train_dict['nb_days']}
train_target_tensor = train_dict['target'] 
train_dataset, train_steps_per_epoch = create_train_tfdata(train_feat_dict,  train_target_tensor, batch_size=512)
  

In [ ]:
#@title BuildModel
def build_model(max_len, item_vocab_size):
    """
    Build a model given the hyper-parameters with item and nb_days input features
    :param hp: (kt.HyperParameters) hyper-parameters to use when building this model
    :return: built and compiled tensorflow model 
    """
    inputs = {}
    inputs['item_id'] = tf.keras.Input(batch_input_shape=[None, max_len],   name='item_id', dtype=tf.int32)
    # create encoding padding mask
    encoding_padding_mask = tf.math.logical_not(tf.math.equal(inputs['item_id'], 0))

    # nb_days bucketized
    inputs['nb_days'] = tf.keras.Input(batch_input_shape=[None, max_len],  name='nb_days', dtype=tf.int32)

    # Pass categorical input through embedding layer
    # with size equals to tokenizer vocabulary size
    # Remember that vocab_size is len of item tokenizer + 1
    # (for the padding '0' value)
    
    embedding_item = tf.keras.layers.Embedding(input_dim=item_vocab_size,
                                               output_dim=16,
                                               name='embedding_item'
                                              )(inputs['item_id'])
    # nbins=100, +1 for zero padding
    embedding_nb_days = tf.keras.layers.Embedding(input_dim=100 + 1,
                                                  output_dim=100,
                                                  name='embedding_nb_days'
                                                 )(inputs['nb_days'])

    #  Concatenate embedding layers
    concat_embedding_input = tf.keras.layers.Concatenate(
     name='concat_embedding_input')([embedding_item, embedding_nb_days])

    concat_embedding_input = tf.keras.layers.BatchNormalization(
     name='batchnorm_inputs')(concat_embedding_input)
    
    # LSTM layer
    rnn = tf.keras.layers.LSTM(units=100,
                                   return_sequences=True,
                                   stateful=False,
                                   recurrent_initializer='glorot_normal',
                                   name='LSTM_cat'
                                   )(concat_embedding_input)

    rnn = tf.keras.layers.BatchNormalization(name='batchnorm_lstm')(rnn)

    # Self attention so key=value in inputs
    att = tf.keras.layers.Attention(use_scale=False, causal=True,
                                    name='attention')(inputs=[rnn, rnn],
                                                      mask=[encoding_padding_mask,
                                                            encoding_padding_mask])

    # Last layer is a fully connected one
    x = tf.keras.layers.Dense(item_vocab_size*2)(att)
    # x = tf.keras.layers.Dense(32)(x)
    output = tf.keras.layers.Dense(item_vocab_size, name='output')(x) #item_vocab_size

    model = tf.keras.Model(inputs, output)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(0.005),
        loss=loss_function,
        metrics=['sparse_categorical_accuracy'])
    
    return model

In [6]:
#@title BuildModelHP
def build_modelhp(max_len, item_vocab_size , embedding_item , embedding_nb_days , rnn_units_cat , learning_rate):
    """
    Build a model given the hyper-parameters with item and nb_days input features
    :param hp: (kt.HyperParameters) hyper-parameters to use when building this model
    :return: built and compiled tensorflow model 
    """
    inputs = {}
    inputs['item_id'] = tf.keras.Input(batch_input_shape=[None, max_len],
                                       name='item_id', dtype=tf.int32)
    # create encoding padding mask
    encoding_padding_mask = tf.math.logical_not(tf.math.equal(inputs['item_id'], 0))

    # nb_days bucketized
    inputs['nb_days'] = tf.keras.Input(batch_input_shape=[None, max_len],
                                       name='nb_days', dtype=tf.int32)

    # Pass categorical input through embedding layer
    # with size equals to tokenizer vocabulary size
    # Remember that vocab_size is len of item tokenizer + 1
    # (for the padding '0' value)
    
    embedding_item = tf.keras.layers.Embedding(input_dim=item_vocab_size,
                                               output_dim=embedding_item,
                                               name='embedding_item'
                                              )(inputs['item_id'])
    # nbins=100, +1 for zero padding
    embedding_nb_days = tf.keras.layers.Embedding(input_dim=100 + 1,
                                                  output_dim=embedding_nb_days,
                                                  name='embedding_nb_days'
                                                 )(inputs['nb_days'])

    #  Concatenate embedding layers
    concat_embedding_input = tf.keras.layers.Concatenate(
     name='concat_embedding_input')([embedding_item, embedding_nb_days])

    concat_embedding_input = tf.keras.layers.BatchNormalization( name='batchnorm_inputs')(concat_embedding_input)
    
    # LSTM layer
    rnn = tf.keras.layers.LSTM(units=rnn_units_cat, return_sequences=True,  stateful=False,recurrent_initializer='glorot_normal', name='LSTM_cat')(concat_embedding_input)

    rnn = tf.keras.layers.BatchNormalization(name='batchnorm_lstm')(rnn)

    # Self attention so key=value in inputs
    att = tf.keras.layers.Attention(use_scale=False, causal=True,
                                    name='attention')(inputs=[rnn, rnn],
                                                      mask=[encoding_padding_mask,
                                                            encoding_padding_mask])

    # Last layer is a fully connected one
    output = tf.keras.layers.Dense(item_vocab_size, name='output')(att)

    model = tf.keras.Model(inputs, output)
    model.compile( optimizer=tf.keras.optimizers.Adam(learning_rate),   loss=loss_function, metrics=['sparse_categorical_accuracy'])
    
    return model

In [7]:
#@title LossFun
def loss_function(real, pred):
    """
    We redefine our own loss function in order to get rid of the '0' value
    which is the one used for padding. This to avoid that the model optimize itself
    by predicting this value because it is the padding one.
    
    :param real: the truth
    :param pred: predictions
    :return: a masked loss where '0' in real (due to padding)
                are not taken into account for the evaluation
    """

    # to check that pred is numric and not nan
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_object_ = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,
                                                                 reduction='none')
    loss_ = loss_object_(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

In [8]:
#@title FitModel
def fit_model(model, train_dataset, steps_per_epoch, epochs):
    """
    Fit the Keras model on the training dataset for a number of given epochs
    :param model: tf model to be trained
    :param train_dataset: (tf.data.Dataset object) the training dataset
                          used to fit the model
    :param steps_per_epoch: (int) Total number of steps (batches of samples) before 
                            declaring one epoch finished and starting the next epoch.
    :param epochs: (int) the number of epochs for the fitting phase
    :return: tuple (mirrored_model, history) with trained model and model history
    """
        # mirrored_strategy allows to use multi GPUs when available
    mirrored_strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy(
        tf.distribute.experimental.CollectiveCommunication.AUTO)
    
    with mirrored_strategy.scope():
        mirrored_model = model

    history = mirrored_model.fit(train_dataset,  steps_per_epoch=steps_per_epoch,     epochs=epochs, verbose=2)

    return mirrored_model, history

In [ ]:
#@title TrainHP
from kerastuner.tuners import RandomSearch

# create a tuner object and define the hyperparameter search space
tuner = RandomSearch(
    build_modelhp,
    objective='val_loss',
    max_trials=10,
    directory='my_dir',
    project_name='my_project',
    hyperparameters={
        'max_len': [3,5,10],
        'item_vocab_size':600,
        'embedding_item': [16, 32, 64],
        'embedding_nb_days': [32, 64, 128],
        'rnn_units_cat': [32, 64, 128],       
        'learning_rate': [0.2, 0.3, 0.4]
    })

# prepare the training and validation data
...

# start the hyperparameter search
tuner.search(train_dataset, epochs=10)

# fit_model(model, train_dataset, steps_per_epoch, epochs)
# print the best hyperparameters found
best_hyperparams = tuner.get_best_hyperparameters()[0].values
print("Best Hyperparameters:", best_hyperparams)


In [99]:
#@title Train

hyperparameters={
        'max_len': max_len,
        'item_vocab_size':item_vocab_size,
        'embedding_item': 16,
        'embedding_nb_days': 16,
        'rnn_units_cat': 128,       
        'learning_rate': 0.008    }


# model=build_model(max_len, item_vocab_size)
model=build_modelhp(**hyperparameters)


fit_model(model, train_dataset,50, 18)

Epoch 1/18
50/50 - 11s - loss: 1.5668 - sparse_categorical_accuracy: 0.3315 - 11s/epoch - 220ms/step
Epoch 2/18
50/50 - 2s - loss: 1.2683 - sparse_categorical_accuracy: 0.4192 - 2s/epoch - 44ms/step
Epoch 3/18
50/50 - 1s - loss: 1.1932 - sparse_categorical_accuracy: 0.4396 - 1s/epoch - 23ms/step
Epoch 4/18
50/50 - 1s - loss: 1.1321 - sparse_categorical_accuracy: 0.4583 - 1s/epoch - 25ms/step
Epoch 5/18
50/50 - 1s - loss: 1.1050 - sparse_categorical_accuracy: 0.4695 - 1s/epoch - 27ms/step
Epoch 6/18
50/50 - 2s - loss: 1.0807 - sparse_categorical_accuracy: 0.4637 - 2s/epoch - 34ms/step
Epoch 7/18
50/50 - 2s - loss: 1.0096 - sparse_categorical_accuracy: 0.4806 - 2s/epoch - 49ms/step
Epoch 8/18
50/50 - 1s - loss: 0.9748 - sparse_categorical_accuracy: 0.4900 - 1s/epoch - 25ms/step
Epoch 9/18
50/50 - 1s - loss: 0.9278 - sparse_categorical_accuracy: 0.5216 - 1s/epoch - 23ms/step
Epoch 10/18
50/50 - 1s - loss: 0.8671 - sparse_categorical_accuracy: 0.5200 - 1s/epoch - 20ms/step
Epoch 11/18
50/5

(<keras.engine.functional.Functional at 0x7f2ed05509d0>,
 <keras.callbacks.History at 0x7f2ed05af820>)

In [ ]:
model.save("/content/drive/MyDrive/Colab Notebook/sanjagh/Recommand/savedmodels/modelRNN")

### Test

In [23]:
servicesdf = pd.read_csv ('/content/drive/MyDrive/Colab Notebooks/sanjagh/Recommand/data/servicesdf.csv', index_col='Unnamed: 0')


In [83]:
def ppredict(userno):
  dftest= df[df.user==userno]
  dfuserservice = pd.merge(dftest, servicesdf, on="service")
  dfuserservice.drop(columns='service', inplace= True)
  print(tabulate(dfuserservice, headers='keys', tablefmt='psql'))


  seqservicet , seqtimet ,seqtargett  =[] ,[] ,[] 
  for user in dftest.user.unique():
    user_data = dftest[dftest['user'] == user].sort_values('time')
    seqservicet.append(user_data.service.values.tolist()[0:-1])
    seqtimet.append(user_data.time.values.tolist()[0:-1])
    seqtargett.append(user_data.time.values.tolist()[1:])

  test_dict = dict(zip(['item_id', 'nb_days', 'target'],[tf.keras.utils.pad_sequences(seqservicet , maxlen=5), tf.keras.utils.pad_sequences(seqtimet, maxlen=5), tf.keras.utils.pad_sequences(seqtargett, maxlen=5)]))
  test_feat_dict = {'item_id': test_dict['item_id'],'nb_days': test_dict['nb_days']}
  y=model.predict(test_feat_dict)
  mmax=0
  mmlist=[]
  for i in range(max_len):
    values= y[0][i].tolist()
    while True:
      if max(values) in mmlist: 
        values[values.index(max(values))]=0
      else:      
        break
    mmax=max(values)
    mmlist.append(mmax)
    print(values.index(max(values)), servicesdf[servicesdf.service==values.index(max(values))].service_title.values)


In [81]:
ppredict(3345)

+----+--------+--------+-----------------------------------+----------+
|    |   user |   time | service_title                     |   subcat |
|----+--------+--------+-----------------------------------+----------|
|  0 |   3345 |      8 | نصب-آیفون-تصویری                  |        1 |
|  1 |   3345 |     56 | تعمیر-و-تعویض-ال-سی-دی-و-تاچ-گوشی |       71 |
+----+--------+--------+-----------------------------------+----------+
1/1 [==============================] - 0s 475ms/step
['بازسازی-خانه']
['ساخت-و-اجرای-نرده-استیل']
['نصب-و-تعمیر-آنتن-دیجیتال']
['نصب-ویندوز-و-نرم-افزار']
['ساخت-و-اجرای-سقف-کاذب']


In [63]:
ppredict(1111)

+----+--------+--------+--------------------------+----------+
|    |   user |   time | service_title            |   subcat |
|----+--------+--------+--------------------------+----------|
|  0 |   1111 |      5 | نصب-و-تعمیر-آنتن-دیجیتال |        1 |
|  1 |   1111 |      5 | تعمیر-تلفن               |        1 |
|  2 |   1111 |      5 | سرویس-و-تعمیر-کولر-آبی   |        0 |
|  3 |   1111 |     91 | هایلایت-و-مش-مو-بانوان   |       68 |
|  4 |   1111 |     91 | پاکسازی-صورت-بانوان      |       38 |
+----+--------+--------+--------------------------+----------+
1/1 [==============================] - 0s 21ms/step
['سرویس-و-تعمیر-یخچال']
['تعمیر-مبل']
['تعمیر-مبل']
['نظافت-منزل']
['قفسه-های-صنعتی-و-فروشگاهی']


In [84]:
ppredict(1111)

+----+--------+--------+--------------------------+----------+
|    |   user |   time | service_title            |   subcat |
|----+--------+--------+--------------------------+----------|
|  0 |   1111 |      5 | نصب-و-تعمیر-آنتن-دیجیتال |        1 |
|  1 |   1111 |      5 | تعمیر-تلفن               |        1 |
|  2 |   1111 |      5 | سرویس-و-تعمیر-کولر-آبی   |        0 |
|  3 |   1111 |     91 | هایلایت-و-مش-مو-بانوان   |       68 |
|  4 |   1111 |     91 | پاکسازی-صورت-بانوان      |       38 |
+----+--------+--------+--------------------------+----------+
1/1 [==============================] - 0s 21ms/step
83 ['بازسازی-خانه']
53 ['نصب-شیشه-سکوریت']
53 ['نصب-شیشه-سکوریت']
88 ['ساخت-و-اجرای-نرده-استیل']
91 ['قفسه-های-صنعتی-و-فروشگاهی']


In [64]:
ppredict(35)

+----+--------+--------+-----------------+----------+
|    |   user |   time | service_title   |   subcat |
|----+--------+--------+-----------------+----------|
|  0 |     35 |      1 | نقاشی-ساختمان   |       80 |
|  1 |     35 |     55 | نصب-پارکت       |        9 |
|  2 |     35 |     55 | شیشه-بری-و-آینه |        7 |
+----+--------+--------+-----------------+----------+
1/1 [==============================] - 0s 20ms/step
['سرویس-و-تعمیر-یخچال']
['نظافت-منزل']
['ساخت-و-اجرای-نرده-استیل']
['نصب-شیشه-سکوریت']
['طراحی-و-ساخت-کابینت-آشپزخانه']


In [85]:
ppredict(35)

+----+--------+--------+-----------------+----------+
|    |   user |   time | service_title   |   subcat |
|----+--------+--------+-----------------+----------|
|  0 |     35 |      1 | نقاشی-ساختمان   |       80 |
|  1 |     35 |     55 | نصب-پارکت       |        9 |
|  2 |     35 |     55 | شیشه-بری-و-آینه |        7 |
+----+--------+--------+-----------------+----------+
1/1 [==============================] - 0s 21ms/step
83 ['بازسازی-خانه']
88 ['ساخت-و-اجرای-نرده-استیل']
75 ['نصب-و-تعمیر-آنتن-دیجیتال']
58 ['نصب-لمینت']
55 ['طراحی-و-ساخت-کابینت-آشپزخانه']


In [69]:
wppredict(1514100)

+----+---------+--------+----------------------+----------+
|    |    user |   time | service_title        |   subcat |
|----+---------+--------+----------------------+----------|
|  0 | 1514100 |     92 | نصب-دوربین-مدار-بسته |        1 |
|  1 | 1514100 |     92 | نصب-دوربین-مدار-بسته |        1 |
+----+---------+--------+----------------------+----------+
1/1 [==============================] - 0s 42ms/step
['سرویس-و-تعمیر-یخچال']
['نظافت-منزل']
['ساخت-و-اجرای-نرده-استیل']
['نصب-چیلر']
['نظافت-منزل']


In [ ]:
df.sort_values('user').iloc[111300:111330]

In [75]:
ppredict(60816)

+----+--------+--------+-----------------------+----------+
|    |   user |   time | service_title         |   subcat |
|----+--------+--------+-----------------------+----------|
|  0 |  60816 |     78 | تعمیر-مبل             |        4 |
|  1 |  60816 |     22 | تعمیرات-موبایل-و-تبلت |       71 |
|  2 |  60816 |     72 | تربیت-سگ              |       33 |
+----+--------+--------+-----------------------+----------+
1/1 [==============================] - 0s 24ms/step
['سرویس-و-تعمیر-یخچال']
['نظافت-منزل']
['ساخت-و-اجرای-نرده-استیل']
['شیشه-بری-و-آینه']
['تعمیر-تلفن']


In [91]:
ppredict(60814)

+----+--------+--------+-----------------------------------+----------+
|    |   user |   time | service_title                     |   subcat |
|----+--------+--------+-----------------------------------+----------|
|  0 |  60814 |     22 | عیب-یابی-و-تعمیرات-سخت-افزار      |       10 |
|  1 |  60814 |     58 | تعمیر-و-تعویض-ال-سی-دی-و-تاچ-گوشی |       71 |
|  2 |  60814 |     80 | نصب-ماشین-ظرفشویی                 |        0 |
+----+--------+--------+-----------------------------------+----------+
1/1 [==============================] - 0s 33ms/step
83 ['بازسازی-خانه']
88 ['ساخت-و-اجرای-نرده-استیل']
75 ['نصب-و-تعمیر-آنتن-دیجیتال']
62 ['بلکا']
58 ['نصب-لمینت']


In [92]:
servicesdf[servicesdf.service_title== 'بلکا']

,service_title,service,subcat
62,بلکا,62,9


In [88]:

ppredict(60818)

+----+--------+--------+-----------------------+----------+
|    |   user |   time | service_title         |   subcat |
|----+--------+--------+-----------------------+----------|
|  0 |  60818 |     22 | تعمیرات-موبایل-و-تبلت |       71 |
|  1 |  60818 |     23 | تعمیرات-موبایل-و-تبلت |       71 |
|  2 |  60818 |     23 | تعمیرات-موبایل-و-تبلت |       71 |
|  3 |  60818 |     23 | تعمیرات-موبایل-و-تبلت |       71 |
|  4 |  60818 |     23 | تعمیرات-موبایل-و-تبلت |       71 |
+----+--------+--------+-----------------------+----------+
1/1 [==============================] - 0s 136ms/step
83 ['بازسازی-خانه']
22 ['ساخت-در-و-پنجره-آهنی']
62 ['بلکا']
62 ['بلکا']
62 ['بلکا']
